# Imports

In [1]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

from sympy import Symbol, solve, integrate # 방정식 해 구하기

# Data settings

## Load dataset
* 데이터셋 불러오기
* `net` : 네트워크 파일
* `OD` : Origin-Destination 수요

In [2]:
data_dir = 'dataset'

In [3]:
network_file = 'Network.csv'
network_path = os.path.join(data_dir, network_file)

In [4]:
OD_file = 'OD.csv'
OD_path = os.path.join(data_dir, OD_file)

In [5]:
network = pd.read_csv(network_path, encoding = 'UTF-8')

In [6]:
OD = pd.read_csv(OD_path, encoding = 'UTF-8')

In [7]:
network.head(3)
#> Init node

,Init node,Term node,Free Flow Time,Capacity,Length,B,Power,Speed.limit,Toll1,Toll2,Type
0,1,2,6,25900.20064,6,0.15,4,0,0,0,1
1,1,3,4,23403.47319,4,0.15,4,0,0,0,1
2,2,1,6,25900.20064,6,0.15,4,0,0,0,1


In [8]:
OD.tail(20).T

,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575
O,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
D,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
T,0,100,100,200,200,800,600,500,700,400,400,300,300,0,100,400,500,1100,700,0


## Preprocessing
* 필요 변수
    * `O` : 기점
    * `D` : 종점
    * `Link_num` : 링크번호
    * `FFT` : Free Flow Time
    * `Alpha`, `Beta` : BPR 함수에 필요한 상수들
    * `Length` : 네트워크 길이
    * `Toll` : 네트워크 요금
    * `Volume` : 링크 통행량
* `BPR Function` : `Cost = Free Flow Time * (1 + (Alpha * (Xn / Capacity))**Beta) + Length * Toll`

In [9]:
def preprocessing(network):
    """네트워크 객체의 컬럼 이름 및 순서를 정리해줌
    - O, D, FFT, alpha, Capacity, beta, Length, Toll 만 남기고 나머지는 지운다.
    """
    # cost를 구한다.
    net = pd.DataFrame({
        'O' : network['Init node '],
        'D' : network['Term node '],
        'FFT' : network['Free Flow Time '],
        'Power' : network['Power'],
        'Capacity' : network['Capacity '],
        'B' : network['B'],
        'Length' : network['Length '],
        #'Toll' : network['Toll '],
        'Link_num' : [i for i in range(1, len(network)+1)],
        'x_n' : 0,
        'y_n' : 0,
        'Direction' : 0,
        'x_n1' : 0,
        'Cost' : 0
    })
   
    net = net.sort_values(by = ['O', 'D']) # net1 데이터프레임을 O, D에 대하여 오름차순 정렬
    net.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return net

# Shortest Path Definition
* `Dijkstra` 방식을 사용하여 각 O-D별 최단경로를 추출하는 함수 생성

## Node Range

In [10]:
def node_range(network):
    """Sioux-Falls Network의 각 기점(Origin)에서 출발하는 링크 번호의 범위를 데이터프레임에 저장하는 함수"""
    origin_nodes = network['O'].unique()
    start_list = []
    end_list = []
    
    for origin in origin_nodes:
        link_list = network[network['O'] == origin]['Link_num'] # 기점이 origin인 모든 링크 번호 리스트 추출
        link_list.sort_values() # 기점 origin에 연결된 링크 번호들을 오름차순으로 정렬
        
        start = link_list.iloc[0] # 링크 리스트의 첫번째 링크(가장 먼저 번호 링크)
        end = link_list.iloc[-1] # 링크 리스트의 마지막 링크(가장 마지막 번호 링크)
        
        start_list.append(start)
        end_list.append(end)
        
    node_link_df = pd.DataFrame({
        'origin' : origin_nodes,
        'start_link' : start_list,
        'end_link' : end_list
    })
    
    node_link_df.set_index('origin', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return node_link_df

## Calculate Cost : $t_a(x_a)$
* 매 iteration마다 갱신되는 $\alpha$, $x_{n}$ 값을 이용해서 Cost를 계산해 주는 함수
* 따라서 iteration을 돌 때마다 다시 계산해주어야 한다.

In [11]:
def calculate_initial_cost(network):
    """
    BPR 함수를 사용하여 링크 통행비용을 계산하는 함수
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost = Free Flow Time * (1 + (B * (X_n / Capacity))**Power) + length * toll
    """
    network['Cost'] = network['FFT'] * (1 + network['B'] * ((network['x_n'] / network['Capacity'])**network['Power'])) #+ (net1['Length'] * net1['Toll'])
    
    return network

In [12]:
def calculate_iterative_cost(network):
    """
    BPR 함수를 사용하여 링크 통행비용을 계산하는 함수
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost = Free Flow Time * (1 + (B * (X_n / Capacity))**Power) + length * toll
    """
    network['Cost'] = network['Cost'] * (1 + network['B'] * ((network['x_n'] / network['Capacity'])**network['Power'])) #+ (net1['Length'] * net1['Toll'])
    
    return network

## Shortest Path Algorithm : Dijkstra

In [13]:
def make_graph(network):
    """네트워크 데이터프레임을 투입하여 graph 객체를 만들어주는 함수"""
    origin_array = np.array(network['O'].unique())
    
    graph = {} # 빈 딕셔너리 생성

    for origin in origin_array:
    
        origin_rows = network[network['O'] == origin]
    
        destin_array = np.array(origin_rows['D'])
        link_num_array = np.array(origin_rows['Link_num'])
        cost_array = np.array(origin_rows['Cost'])
    
        tup_dict = {}
    
        for destin, link_num, cost in zip(destin_array, link_num_array, cost_array):
            #tup_list.append((destin, link_num, cost))
            tup_dict[destin] = (link_num, cost)
    
        graph[origin] = tup_dict
    
    return graph

In [14]:
def Dijkstra(network, initial, final):
    
    graph = make_graph(network)
    
    path = {} #shortest path
    adj_node = {} #neighbouring nodes
    
    queue = [] #Queue for manipulation
    
    #Check all nodes and initialize path with 0
    for node in graph:
        path[node] = float("inf")
        adj_node[node] = None
        queue.append(node)

    path[initial] = 0
    #Check visited nodes and also to find the minimum distance between the nodes.
    while queue:
        key_min = queue[0]
        min_val = path[key_min]
        for n in range(1, len(queue)):
            if path[queue[n]] < min_val:
                key_min = queue[n]
                min_val = path[key_min]
        cur = key_min
        queue.remove(cur)

        for i in graph[cur]:
            alternate = graph[cur][i][1] + path[cur]
            if path[i] > alternate:
                path[i] = alternate
                
                adj_node[i] = cur
                
    nodes = [final]
    #Finally, print nodes that satisfies the condition
    #target = final
    
    while True:
        final = adj_node[final]
        if final is None:
            break
        nodes.append(final)
    
    nodes.reverse()
    
    #costs = list(path.values())[target]

    links = []
    
    for i in range(len(nodes)-1):
        link = graph[nodes[i]][nodes[i+1]][0]
        links.append(link)
        
    return links #,nodes # 각 start-end별 최저비용(cost) 및 해당 구간당 경유링크, (경유노드) 반환

# Initialization
* **`N = 0`**
* 초기 비용 계산 : $x_0$ = 0 (모든 링크의 통행량 = 0)인 상태에서 비용 계산
* 초기 배정 : All-or-Nothing 방식으로 통행 배정, $x_1$ 획득

## Data Ready

In [15]:
net1 = preprocessing(network)

In [16]:
net1.head(3)

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.20064,0.15,6,1,0,0,0,0,0
2,1,3,4,4,23403.47319,0.15,4,2,0,0,0,0,0
3,2,1,6,4,25900.20064,0.15,6,3,0,0,0,0,0


## 초기 비용 계산 : $t_a(x_a^0)$
* $x_0$ = 0 (전부 0)임을 계산하여 Cost 계산

In [17]:
net_cost = calculate_initial_cost(net1) # x0 

In [18]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,0,0,0,0,6.0
2,1,3,4,4,23403.473190,0.15,4,2,0,0,0,0,4.0
3,2,1,6,4,25900.200640,0.15,6,3,0,0,0,0,6.0
4,2,6,5,4,4958.180928,0.15,5,4,0,0,0,0,5.0
5,3,1,4,4,23403.473190,0.15,4,5,0,0,0,0,4.0


## $x_1$ 획득 : AON Assignment

In [ ]:
# 각 OD별로 Dijkstra를 사용해 최단경로를 산출, 
# OD 테이블의 교통량(T)을 각 최단경로에 포함된 링크에 All-or-Nothing으로 배정하기

origin_list = OD['O'].unique()

for origin in tqdm(origin_list): # 각 Origin에 대하여
    
    #print(f'-------- origin = {origin} --------')
    
    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D'].unique()
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            link_list = Dijkstra(net_cost, origin, destination)
            #print(f'(O, D) = ({origin}, {destination}), Path link list = {link_list}')
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'x_n'] += add_volume # All or Nothing 배정
            #print(f'(O, D) = ({origin}, {destination}), Path Link_num list = {link_list}, Add_volume = {add_volume}')
        else:
            pass

In [20]:
net_cost

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,3800,0,0,0,6.0
2,1,3,4,4,23403.473190,0.15,4,2,6000,0,0,0,4.0
3,2,1,6,4,25900.200640,0.15,6,3,3800,0,0,0,6.0
4,2,6,5,4,4958.180928,0.15,5,4,6600,0,0,0,5.0
5,3,1,4,4,23403.473190,0.15,4,5,6000,0,0,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,23,22,4,2,5000.000000,0.25,4,72,10800,0,0,0,4.0
73,23,24,2,2,5078.508436,0.25,2,73,5700,0,0,0,2.0
74,24,13,4,2,5091.256152,0.25,4,74,10800,0,0,0,4.0


# Iteration

## $t_1$ 계산 : 통행비용 갱신
* Update Link Travel Time
* 전 단계에서 계산한 통행량 $x_1$로 통행비용(Cost) 갱신

In [21]:
net_cost = calculate_iterative_cost(net_cost) # Cost 일부 갱신

In [22]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,3800,0,0,0,6.000417
2,1,3,4,4,23403.473190,0.15,4,2,6000,0,0,0,4.002592
3,2,1,6,4,25900.200640,0.15,6,3,3800,0,0,0,6.000417
4,2,6,5,4,4958.180928,0.15,5,4,6600,0,0,0,7.354765
5,3,1,4,4,23403.473190,0.15,4,5,6000,0,0,0,4.002592


## $y_1$ 획득 : AON Assignment
* 전 단계의 $t_1$을 이용, Dijkstra로 Shortest Path 계산
* All or Nothing Assignment 실행, y1 획득

In [ ]:
for origin in tqdm(origin_list): # 각 Origin에 대하여
    
    #print(f'origin = {origin}')
    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D'].unique()
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            link_list = Dijkstra(net_cost, origin, destination)
            #print(f'destination = {destination}, link list = {link_list}')
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'y_n'] += add_volume # All or Nothing 배정 :: y0 갱신
            #print(f'(O, D) = ({origin}, {destination}), Path Link_num list = {link_list}, Add_volume = {add_volume}')
        else:
            pass

In [24]:
net_cost

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,3800,4900,0,0,6.000417
2,1,3,4,4,23403.473190,0.15,4,2,6000,13000,0,0,4.002592
3,2,1,6,4,25900.200640,0.15,6,3,3800,5000,0,0,6.000417
4,2,6,5,4,4958.180928,0.15,5,4,6600,1700,0,0,7.354765
5,3,1,4,4,23403.473190,0.15,4,5,6000,12900,0,0,4.002592
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,23,22,4,2,5000.000000,0.25,4,72,10800,7600,0,0,8.665600
73,23,24,2,2,5078.508436,0.25,2,73,5700,18600,0,0,2.629865
74,24,13,4,2,5091.256152,0.25,4,74,10800,16200,0,0,8.499846


# Calculate $Z(x)$

In [54]:
def calculate_Z(network):
    """네트워크 파일로부터 Z(x) 값 계산하기"""
    ## Sympy :: 방정식을 풀기 위한 패키지 이용. 방정식에 포함될 각 변수들을 선언하기

    d = Symbol('d') # direction
    B = Symbol('B') # B. 0.15 있는 그거
    t = Symbol('t') # t. Cost
    c = Symbol('c') # capacity
    x = Symbol('x') # x0
    alpha = Symbol('alpha') # alpha
    P = Symbol('P') # Power. 4.
    
    z_x = 0

    for i in tqdm(range(1, len(net_cost)+1)): # 1부터 끝까지 Link_num == i에 대하여
        t = network.loc[i, 'FFT']
        B = network.loc[i, 'B']
        P = network.loc[i, 'Power']
        C = network.loc[i, 'Capacity']
    
        equation = t * (1 + B * ((x/C)**P))
    
        xa = network.loc[i, 'x_n']

        integ = integrate(equation, (x, 0, xa))
    
        z_x += integ
    
    return z_x

In [55]:
calculate_Z(net_cost)

100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [00:01<00:00, 71.38it/s]


8372244.14651346

## Find Move Size $\alpha$
* Finding Direction : $d = y_1-x_1$ 
* 목적함수 $Z(x)$를 최소화하는 $\alpha$ 값 찾기
* 즉, $\sum(y_a^n-x_a^n)*t_a*(x_a^n+\alpha(y_a^n-x_a^n) = 0$이 되는 $\alpha$ 값 찾기

In [25]:
net_cost['Direction'] = net_cost['y_n'] - net_cost['x_n'] # direction :: d = yn - xn 임

In [27]:
net_cost.head(3)

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.20064,0.15,6,1,3800,4900,1100,0,6.000417
2,1,3,4,4,23403.47319,0.15,4,2,6000,13000,7000,0,4.002592
3,2,1,6,4,25900.20064,0.15,6,3,3800,5000,1200,0,6.000417


In [28]:
z_x = 0 # Z(x)

for i in tqdm(range(1, len(net_cost)+1)): # 1부터 끝까지 Link_num == i에 대하여
    
    d = net_cost.loc[i, 'Direction'] # i링크의 direction
    t = net_cost.loc[i, 'Cost']
    B = net_cost.loc[i, 'B']
    x = net_cost.loc[i, 'x_n']
    P = net_cost.loc[i, 'Power']
    C = net_cost.loc[i, 'Capacity']
    
    equation = d * t * (1 + B * (((x + alpha * d)/C)**P))
    #print(equation)
    
    total_equation += equation

100%|█████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 242.53it/s]


In [29]:
alpha_list = solve(total_equation) # 여기서 alpha는 0과 1 사이의 양수여야 하므로, 해당 부분만을 필터링해준다.
print(alpha_list)

[-21.2811398646103, 0.704914863110394, 0.82108598205148 - 0.589367242502786*I, 0.82108598205148 + 0.589367242502786*I]


In [30]:
def alpha_range(x):
    """Alpha 값 중 0 이상 1 이하의 값만 반환하게끔 하는 함수"""
    decision = (x <= 1) and (x >= 0)
    return decision

In [31]:
def find_alpha(alpha_list):
    """Alpha 값들 중 0 이상 1 이하 실수값만을 반환"""
    for alpha in alpha_list:
        if alpha.is_real == True:
            if alpha <= 1 and alpha >= 0:
                #print(alpha)
                return alpha
            else:
                pass
        else:
            pass

In [32]:
ith_alpha = find_alpha(alpha_list)
print(ith_alpha) #### 금번 iteration의 alpha를 구했다.

0.704914863110394


## $x_2$ 계산
* $\alpha$ 값을 이용하여 $x_2$ 값을 계산
* $x_2 = x_1 + \alpha*(y_1 - x_1)$

In [33]:
net_cost['x_n1'] = net_cost['x_n'] + ith_alpha * (net_cost['y_n'] - net_cost['x_n'])

In [34]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,3800,4900,1100,4575.40634942143,6.000417
2,1,3,4,4,23403.473190,0.15,4,2,6000,13000,7000,10934.4040417728,4.002592
3,2,1,6,4,25900.200640,0.15,6,3,3800,5000,1200,4645.89783573247,6.000417
4,2,6,5,4,4958.180928,0.15,5,4,6600,1700,-4900,3145.91717075907,7.354765
5,3,1,4,4,23403.473190,0.15,4,5,6000,12900,6900,10863.9125554617,4.002592


## Convergence Test
* 컨버전스 테스트 실행
* 만약 컨버전스 테스트를 통과하지 못할 시, 이 $x_2$를 이용해서 iteration을 반복하게 된다.

In [35]:
def convergence_test(network, k):
    """네트워크에 대한 convergence test를 실행하는 함수.
    :: network :: 네트워크 데이터프레임. x_n, x_n1, Cost 열을 포함하고 있어야 함
    :: k :: 수렴 기준. 0.000001 정도로 잡아 본다."""
    sum_xn = sum(network['x_n'] * network['Cost']) # 분모
    sum_varx = sum((network['x_n'] - network['x_n']) ** 2) # 분자
    
    criterion = sum_varx / sum_xn
        
    if criterion <= k: # 수렴기준 충족
        return True
    
    else:
        return False

In [36]:
convergence = convergence_test(net_cost, 0.000001) # False가 저장될 시, 위 과정을 반복한다.
print(convergence)

if convergence == False:
    net_cost['x_n'] = net_cost['x_n1']
    net_cost['y_n'] = 0 # 초기화해주지 않으면 엄청 늘어나버린다.
    net_cost['Direction'] = 0
    
else:
    pass

False
